In [19]:
import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


In [79]:
# Create the database connection
con = duckdb.connect(".config/nfl.duckdb")
#con.close()

In [ ]:
# Creating dataframes with DuckDB, plays and player_play both have 50 columns, more ideal for a broad random forest
X = con.sql("""
    SELECT quarter, down, yardsToGo, yardlineNumber, preSnapHomeScore, preSnapVisitorScore,
    playNullifiedByPenalty, absoluteYardlineNumber, preSnapHomeTeamWinProbability, preSnapVisitorTeamWinProbability, expectedPoints,
    passResult_complete, passResult_incomplete, passResult_sack, passResult_interception, passResult_scramble, passLength, targetX, targetY,
    playAction, passTippedAtLine, unblockedPressure, qbSpike, qbKneel, qbSneak, penaltyYards, prePenaltyYardsGained, 
    homeTeamWinProbabilityAdded, visitorTeamWinProbilityAdded, expectedPointsAdded, isDropback, timeToThrow, timeInTackleBox, timeToSack,
    dropbackDistance, pff_runPassOption, playClockAtSnap, pff_manZone, pff_runConceptPrimary_num, pff_passCoverage_num, pff_runConceptSecondary_num
FROM bronze.plays
""").df()
y = con.sql("""
    SELECT yardsGained
    FROM bronze.plays
""").df()

In [95]:
con.sql("SELECT * FROM X LIMIT 10").df().describe()

#pff_runConceptPrimary, pff_runConceptSecondary, pff_passCoverage

,quarter,down,yardsToGo,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,playNullifiedByPenalty,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,...,unblockedPressure,qbSpike,qbKneel,qbSneak,prePenaltyYardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runPassOption
count,10.000000,10.000000,10.000000,10.000000,10.0000,10.000000,10.0,10.000000,10.000000,10.000000,...,10.0,10.0,10.0,10.0,10.000000,10.000000,10.000000,10.000000,10.0,10.0
mean,3.200000,2.100000,9.800000,26.600000,15.6000,16.600000,0.0,41.000000,0.446718,0.553282,...,1.0,1.0,1.0,1.0,4.400000,0.007710,-0.007710,-0.122348,1.0,0.0
std,1.032796,0.875595,1.988858,9.203864,11.3842,12.249263,0.0,17.088007,0.396046,0.396046,...,0.0,0.0,0.0,0.0,4.903513,0.022025,0.022025,0.965255,0.0,0.0
min,1.000000,1.000000,6.000000,8.000000,0.0000,0.000000,0.0,18.000000,0.005253,0.002189,...,1.0,1.0,1.0,1.0,-1.000000,-0.024210,-0.058490,-1.161621,1.0,0.0
25%,3.000000,1.250000,8.500000,21.500000,7.0000,7.750000,0.0,31.500000,0.093530,0.201843,...,1.0,1.0,1.0,1.0,0.750000,-0.000930,-0.010429,-0.589813,1.0,0.0
50%,3.500000,2.000000,10.000000,27.500000,15.5000,17.000000,0.0,38.000000,0.417364,0.582636,...,1.0,1.0,1.0,1.0,4.000000,0.002518,-0.002518,-0.334129,1.0,0.0
75%,4.000000,3.000000,11.500000,33.500000,23.7500,23.750000,0.0,45.000000,0.798157,0.906470,...,1.0,1.0,1.0,1.0,5.750000,0.010429,0.000930,-0.054620,1.0,0.0
max,4.000000,3.000000,12.000000,40.000000,35.0000,38.000000,0.0,82.000000,0.997811,0.994747,...,1.0,1.0,1.0,1.0,15.000000,0.058490,0.024210,2.194760,1.0,0.0


In [81]:
con.sql("SELECT DISTINCT pff_runConceptPrimary FROM bronze.plays")

┌───────────────────────┐
│ pff_runConceptPrimary │
│        varchar        │
├───────────────────────┤
│ POWER                 │
│ PULL LEAD             │
│ MAN                   │
│ INSIDE ZONE           │
│ TRAP                  │
│ DRAW                  │
│ TRICK                 │
│ NA                    │
│ COUNTER               │
│ OUTSIDE ZONE          │
│ FB RUN                │
│ SNEAK                 │
│ UNDEFINED             │
├───────────────────────┤
│        13 rows        │
└───────────────────────┘

In [ ]:
# Having issues with NA values, the below code does a simple count using pandas, will then go back and change the query
na_counts = (X == 'NA').sum()

# Optionally, filter only columns with 'NA' values for easier review
na_counts_filtered = na_counts[na_counts > 0]
print(na_counts_filtered) # playClockAtSnap has only 1 NA value, will just drop that row


playClockAtSnap         1
passLength           7398
targetX              7748
targetY              7748
penaltyYards        15740
timeToThrow          7419
timeInTackleBox      7207
timeToSack          15516
dropbackDistance     5966
pff_manZone           192
dtype: int64


In [30]:
# Instantiate the model and split the data
rf = RandomForestRegressor()

selector = RFE(rf, n_features_to_select=10, step=1)
X_selected = selector.fit_transform(X, y)


ValueError: could not convert string to float: 'NA'